In [ ]:
#please enter your project details in here
env = 'dev'
project_name = "BUName_usecase"
job_name = 'tr'

In [ ]:
import os
import pickle
import boto3
import gzip
from io import BytesIO
#import joblib
import sagemaker
import sagemaker.session

from sagemaker.transformer import Transformer
from sagemaker.estimator import Estimator
from sagemaker.inputs import TrainingInput,TransformInput,CreateModelInput
from sagemaker.processing import (
    ProcessingInput,
    ProcessingOutput,
    ScriptProcessor,
)
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.workflow.conditions import (
    ConditionGreaterThanOrEqualTo,
)
from sagemaker.workflow.condition_step import (
    ConditionStep,
    JsonGet,
)
from sagemaker.model_metrics import (
    MetricsSource,
    ModelMetrics,
)
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
)
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.properties import PropertyFile
from sagemaker.workflow.steps import (
    ProcessingStep,
    TrainingStep,
    TransformStep,
    CreateModelStep
)
from sagemaker.workflow.step_collections import RegisterModel
from sagemaker.model import Model
from sagemaker.network import NetworkConfig

from datetime import datetime
from dateutil import tz
import json
import boto3
from sagemaker.workflow.parameters import (
        ParameterInteger,
        ParameterString,
    )

sess = sagemaker.Session()

import pytz
from datetime import datetime

def convert_timezone():
    to_zone = tz.gettz('Asia/Kolkata')
    
    return datetime.utcnow().astimezone(to_zone)

   
def get_session(region, default_bucket):
    """Gets the sagemaker session based on the region.
    Args:
        region: the aws region to start the session
        default_bucket: the bucket to use for storing the artifacts
    Returns:
        `sagemaker.session.Session instance
    """

    boto_session = boto3.Session(region_name=region)

    sagemaker_client = boto_session.client("sagemaker")
    runtime_client = boto_session.client("sagemaker-runtime")
    return sagemaker.session.Session(
        boto_session=boto_session,
        sagemaker_client=sagemaker_client,
        sagemaker_runtime_client=runtime_client,
        default_bucket=default_bucket,
    )

def latest_config():
        
        '''
        this will pick the latest trained config file
        '''

        s3_path = f"s3://dlk-cloud-tier-8-code-ml-{env}/"
        config_bucket = f"dlk-cloud-tier-8-code-ml-{env}"
        s3 = boto3.client('s3')
        all_objects = s3.list_objects(Bucket = f"dlk-cloud-tier-8-code-ml-{env}", Prefix=f"config_files/pipeline_config/{project_name}/")
        model_folder=all_objects['Contents'][-1]['Key']
        #model_path=s3_path+model_folder

        return config_bucket, model_folder

config_s3_bucket = latest_config()[0]
model_s3_path = latest_config()[1]


def getJsonData(bucket_name,key_name):
        '''
        this will pick the json config file from s3 bucket
        '''

        print("[LOG]", bucket_name,'---------')
        print("[LOG]", key_name,'--------------')

        s3_client = boto3.client('s3')
        csv_obj = s3_client.get_object(Bucket=bucket_name, Key=key_name)

        body = csv_obj['Body']

        json_string = body.read().decode('utf-8')
        json_content = json.loads(json_string)

        return json_content


config = getJsonData(config_s3_bucket,model_s3_path)

## Preprocessing, Training & Register Model

In [10]:
def get_pipeline(
    region,
    subnets,
    security_group_ids,
    role=None,
    default_bucket=None,
    model_package_group_name=config['pipeline']['pipeline_config']['model_package_group_name'],
    pipeline_name=config['pipeline']['pipeline_config']['pipeline_name'],  # You can find your pipeline name in the Studio UI (project -> Pipelines -> name)
    base_job_prefix=config['pipeline']['pipeline_config']['base_job_prefix'],
    env=config['pipeline']['pipeline_config']['env']
):
    
    
    srilanka_tz = pytz.timezone('Asia/Colombo')
    s3 = boto3.client('s3')
    #working input data path
    input_data = config['tr_preprocessing']['inputs']['input1']
    #input_data2 = config['tr_preprocessing']['inputs']['input2']
    #input_data3 = config['tr_preprocessing']['inputs']['input3']
    #input_data4 = config['tr_preprocessing']['inputs']['input4']
    
    directory_name = config['pipeline']['pipeline_config']['env']
    date_folder = datetime.now(srilanka_tz).strftime("%Y-%m-%d")
       
    sagemaker_session = get_session(region, default_bucket)
    if role is None:
        role = sagemaker.session.get_execution_role(sagemaker_session)
    account_id = boto3.client("sts").get_caller_identity().get("Account")
    region = boto3.session.Session().region_name

    # Parameters for pipeline execution
    
    preprocess_output_data1 = f"s3://dlk-cloud-tier-10-preprocessed-ml-{env}/{project_name}/{job_name}/{date_folder}/preprocess_output_data/output1"
    #preprocess_output_data2 = f"s3://dlk-cloud-tier-10-preprocessed-ml-{env}{project_name}/{job_name}/{date_folder}/preprocess_output_data/output2"
    #preprocess_output_data3 = f"s3://dlk-cloud-tier-10-preprocessed-ml-{env}/{project_name}/{job_name}/{date_folder}/preprocess_output_data/output3"
    #preprocess_output_data4 = f"s3://dlk-cloud-tier-10-preprocessed-ml-{env}/{project_name}/{job_name}/{date_folder}/preprocess_output_data/output4"
    
    model_path = ParameterString(
        name="ModelPath",
        default_value=f"s3://dlk-cloud-tier-9-training-ml-{env}/mobile_prepaidchurn/models/{date_folder}", 
    )
        
    model_approval_status = ParameterString(
        name="ModelApprovalStatus",
        default_value="Approved",  # ModelApprovalStatus can be set to a default of "Approved" if you don't want manual approval.
    )     
 
    generic_tags=[{'Key': 'EnvironmentName', 'Value': env}, 
                  {'Key': 'ProjectName', 'Value': config['pipeline']['generic_tags']['ProjectName']},
                  {'Key': 'DepartmentName', 'Value': config['pipeline']['generic_tags']['DepartmentName']}, 
                  {'Key': 'UsecaseName', 'Value': config['pipeline']['generic_tags']['UsecaseName']},
                  {'Key': 'ResourceName', 'Value': config['pipeline']['generic_tags']['ResourceName']}, 
                  {'Key': 'OwnerName', 'Value': config['pipeline']['generic_tags']['OwnerName']},
                  {'Key': 'BUName', 'Value': config['pipeline']['generic_tags']['BUName']} ]
    

    # --------------------- PREPROCESSING --------------------------------------------------------------------
    ecr_repository = config['tr_preprocessing']['estimator_config']['ecr_repository']
    tag = ":latest"
    uri_suffix = "amazonaws.com"
    
    preprocessing_repository_uri = "{}.dkr.ecr.{}.{}/{}".format(
        account_id, region, uri_suffix, ecr_repository + tag
    )
        
    script_processor = ScriptProcessor(
         command = ["python3"],
         image_uri = preprocessing_repository_uri,
         role = role,
         instance_count = 1,
         instance_type = "ml.m5.2xlarge",
         tags = generic_tags + [{'Key': 'JobType', 'Value': 'Preprocessing'}],
         network_config = NetworkConfig(subnets=subnets.split(':'), security_group_ids=security_group_ids.split(':'))
    )
    
    step_preprocess = ProcessingStep(
        name=f"{base_job_prefix}-preprocessing",
        processor=script_processor, 
        code=config['tr_preprocessing']['inputs']['code'],
        inputs=[ProcessingInput(input_name="input-1",source=input_data, destination="/opt/ml/processing/input"),
               #ProcessingInput(input_name="input-2", source=input_data2, destination="/opt/ml/processing/input2"),
                #ProcessingInput(input_name="input-3", source=input_data3, destination="/opt/ml/processing/input3"),
                #ProcessingInput(input_name="input-4", source=input_data4, destination="/opt/ml/processing/input4"),
               ],
        outputs=[
            ProcessingOutput(output_name="output1", destination=preprocess_output_data1, source="/opt/ml/processing/output1"),
            #ProcessingOutput(output_name="output2", destination=preprocess_output_data2, source="/opt/ml/processing/output2"),
            #ProcessingOutput(output_name="output3", destination=preprocess_output_data3, source="/opt/ml/processing/output3"),
            #ProcessingOutput(output_name="output4", destination=preprocess_output_data4, source="/opt/ml/processing/output4"),
        ]
    )
    
    
    ##--------------------------------------- Training Job-----------------------------------------------------
    
    ecr_repository = config['train_model']['estimator_config']['ecr_repository']
    tag = ":latest"
    uri_suffix = "amazonaws.com"
    
    recommender_image_uri = "{}.dkr.ecr.{}.{}/{}".format(
        account_id, region, uri_suffix, ecr_repository + tag
    )
    
    estimator = Estimator(image_uri=recommender_image_uri,
                      role=role,
                      sagemaker_session=sess,
                      output_path=model_path,
                      base_job_name=f"{base_job_prefix}-training-job",
                      input_mode='File',                                     
                      train_instance_count=config['train_model']['estimator_config']['instance_count'],
                      train_instance_type=config['train_model']['estimator_config']['instance_type'],
                      debugger_hook_config=False,
                      disable_profiler = True,
                      metric_definitions=[
                          # You can define your own regex
                            {'Name': 'Accuracy: ' , 'Regex': 'Accuracy: ([-+]?[0-9]*\.?[0-9]+)'},
                            {'Name': 'Precision: ' , 'Regex': 'Precision: ([-+]?[0-9]*\.?[0-9]+)'},
                            {'Name': 'Recall: ' , 'Regex': 'Recall: ([-+]?[0-9]*\.?[0-9]+)'},
                            {'Name': 'F1 score: ' , 'Regex': 'f1: ([-+]?[0-9]*\.?[0-9]+)'},
                            {'Name': 'Latency: ' , 'Regex': 'Latency: ([-+]?[0-9]*\.?[0-9]+)'},
                      ],
                      subnets = subnets.split(':'),
                      security_group_ids = security_group_ids.split(':')
                         )

    # start training
    step_train = TrainingStep(
        name=f"{base_job_prefix}-training",
        estimator=estimator,
        inputs = {"training": TrainingInput(
                s3_data=step_preprocess.properties.ProcessingOutputConfig.Outputs["output1"].S3Output.S3Uri,
                content_type="text/csv",
            ),
                 # "training2": TrainingInput(
                #s3_data=step_preprocess.properties.ProcessingOutputConfig.Outputs["output2"].S3Output.S3Uri,
                #content_type="text/csv",
            #),
                   # "training3": TrainingInput(
                #s3_data=step_preprocess.properties.ProcessingOutputConfig.Outputs["output3"].S3Output.S3Uri,
                #content_type="text/csv",
            #),
                   # "training4": TrainingInput(
                #s3_data=step_preprocess.properties.ProcessingOutputConfig.Outputs["output4"].S3Output.S3Uri,
                #content_type="text/csv",
            #),
        }
    )

    ###### --------------------- Model Registry ----------------------------------------------------------------
    
    #registering the model

    step_register = RegisterModel(
        name= f"{base_job_prefix}-registerxgbmodel",
        estimator= estimator,
        model_data= step_train.properties.ModelArtifacts.S3ModelArtifacts,
        content_types= ["text/csv"],
        response_types= ["text/csv"],
        inference_instances= ["ml.t2.medium", "ml.m5.xlarge"],
        transform_instances= ["ml.m5.xlarge"],
        model_package_group_name=f"{model_package_group_name}-XGB",
        approval_status=model_approval_status,
        #model_metrics=model_metrics,
    )

    # Pipeline instance
    pipeline = Pipeline(
        name=pipeline_name+env,
        parameters=[
            env,
            model_approval_status,
            model_path
        ],
        steps=[step_preprocess,
               step_train,
               step_register
              ],
        sagemaker_session=sagemaker_session,
    )
    return pipeline

In [11]:
#new subnet
dev_subnets = 'subnet-036d6b39301b4a41a'
dev_sg = 'sg-0970930479bbef573'

model_package_group_name=config['pipeline']['pipeline_config']['model_package_group_name']
pipeline_name=config['pipeline']['pipeline_config']['pipeline_name'] # You can find your pipeline name in the Studio UI (project -> Pipelines -> name)
base_job_prefix=config['pipeline']['pipeline_config']['base_job_prefix']
env=config['pipeline']['pipeline_config']['env']

region ='ap-southeast-1'
role='arn:aws:iam::120582440665:role/Sagemaker'
default_bucket='pipeline-sagemaker-test'
pipeline_def = get_pipeline(
    region=region,
    subnets = dev_subnets,
    security_group_ids = dev_sg,
    role=role,
    default_bucket=default_bucket,
    model_package_group_name=model_package_group_name,
    pipeline_name=pipeline_name,
    env='dev',
)

pipeline_def.upsert(role_arn=role)

execution = pipeline_def.start()

In [12]:
execution.describe()

{'PipelineArn': 'arn:aws:sagemaker:ap-southeast-1:120582440665:pipeline/mobile-prepaidchurn-tr-pipeline-dev',
 'PipelineExecutionArn': 'arn:aws:sagemaker:ap-southeast-1:120582440665:pipeline/mobile-prepaidchurn-tr-pipeline-dev/execution/zv0cjrvtf1gd',
 'PipelineExecutionDisplayName': 'execution-1647313891021',
 'PipelineExecutionStatus': 'Executing',
 'CreationTime': datetime.datetime(2022, 3, 15, 3, 11, 30, 952000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2022, 3, 15, 3, 11, 30, 952000, tzinfo=tzlocal()),
 'CreatedBy': {},
 'LastModifiedBy': {},
 'ResponseMetadata': {'RequestId': '84397e9d-1984-4970-8210-cf2de565b6d3',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '84397e9d-1984-4970-8210-cf2de565b6d3',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '445',
   'date': 'Tue, 15 Mar 2022 03:11:30 GMT'},
  'RetryAttempts': 0}}

------------------------